In [1]:
!pip install transformers underthesea torch numpy pandas json sklearn

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import json
import os
import csv 
from sklearn.metrics.pairwise import cosine_similarity
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base")

c:\Users\quang\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open(r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [7]:
def get_embedding(text, qid=None, subject=None, max_len=512):
    if not text.strip():
        return np.zeros(768)
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_len)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # CLS
    except Exception as e:
        print(f"Lỗi khi embedding câu hỏi id={qid}, môn={subject}")
        return np.zeros(768)

Đoạn code dưới đây sẽ trích xuất ra độ nhiễu của giữa từng options so với answer:   
- **mean_sim**: trung bình cosine similarity giữa đáp án đúng và các đáp án sai. Càng cao càng giống
- **max_sim**: giá trị similarity lớn nhất giữa đáp án đúng và các đáp án sai  
- **min_sim**: giá trị similarity nhỏ nhất  
- **std_sim**: độ lệch chuẩn similarity. Nếu std_sim cao → có option gần đúng và option rất xa đúng cùng tồn tại, câu hỏi có thể gây phân vân; nếu thấp → tất cả option sai đều gần như cùng mức so với đáp án đúng.
- **range_sim**: khoảng cách giữa max_sim và min_sim. một số option rất gần đáp án đúng, số khác rất xa, gây phân tán độ khó.


In [12]:
features = []
ids = []
csv_rows = []

for item in data:
    question = item["question"]
    options = item["options"]
    answer_key = item["answer"]

    try:
        answer_index = options.index(answer_key)
    except ValueError:
        print(f"Câu hỏi {item['id']} không có answer trong options")
        continue 

    emb_answer = get_embedding(question + " " + options[answer_index], qid=item["id"], subject=item["subject"])
    if emb_answer is None or not emb_answer.any():
        continue

    wrong_embs = [
        get_embedding(question + " " + opt, qid=item["id"], subject=item["subject"])
        for i, opt in enumerate(options) if i != answer_index
    ]
    sims = [cosine_similarity([emb_answer], [emb])[0][0] for emb in wrong_embs]

    if len(sims) > 0:
        mean_sim = float(np.mean(sims))
        max_sim = float(np.max(sims))
        min_sim = float(np.min(sims))
        std_sim = float(np.std(sims))
        range_sim = max_sim - min_sim
    else:
        mean_sim = max_sim = min_sim = std_sim = range_sim = 0.0

    features.append([mean_sim, max_sim, min_sim, std_sim, range_sim])
    ids.append(item["id"])
    csv_rows.append({
        "id": item["id"],
        "question": item["question"],
        "answer": answer_key,
        "mean_sim": mean_sim,
        "max_sim": max_sim,
        "min_sim": min_sim,
        "std_sim": std_sim,
        "range_sim": range_sim
    })
features = np.array(features) 
ids = np.array(ids)

Lỗi khi embedding câu hỏi id=59, môn=Văn_học
Lỗi khi embedding câu hỏi id=60, môn=Văn_học


In [13]:
# ===== 5. Lưu numpy =====
output_dir = r"D:\NCKH\estimate_question_difficulty\Embedding\Output_features"
os.makedirs(output_dir, exist_ok=True)

features = np.array(features)
ids = np.array(ids)

np.save(os.path.join(output_dir, "noise_features.npy"), features)
np.save(os.path.join(output_dir, "ids.npy"), ids)
print("Đã lưu noise_features.npy và ids.npy")

# ===== 6. Lưu CSV =====
csv_path = os.path.join(output_dir, "noise_features.csv")
with open(csv_path, "w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=csv_rows[0].keys())
    writer.writeheader()
    writer.writerows(csv_rows)

print("Đã lưu CSV kiểm tra:", csv_path)

Đã lưu noise_features.npy và ids.npy
Đã lưu CSV kiểm tra: D:\NCKH\estimate_question_difficulty\Embedding\Output_features\noise_features.csv
